In [64]:
import pickle
import numpy as np
import open3d as o3d
from laserscan import SemLaserScan
import yaml
import matplotlib.pyplot as plt
import copy
import trimesh
import os

In [66]:
def save_pc(pcd_list, filename="pc.png"):
    vis = o3d.visualization.Visualizer()
    vis.create_window(visible=False) 
    vis.add_geometry(pcd)
    vis.update_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()
    vis.capture_screen_image(filename, do_render=True)
    vis.destroy_window()

In [67]:
def display_inlier_outlier(cloud, ind, window_name="Open3d"):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud], window_name=window_name)

In [68]:
load_kitti = False

## KITTI load

In [69]:
with open("semantic_kitti/semantic-kitti-all.yaml", "r") as stream:
    try:
        CFG = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [70]:
kitti_fname = "000000"
seq_no = 0
"/{:02d}/".format(seq_no)

'/00/'

In [71]:
if load_kitti:
    scan = SemLaserScan(sem_color_dict=CFG['color_map'])
    scan.open_scan("semantic_kitti/dataset/sequences/{:02d}/velodyne/{}.bin".format(seq_no, kitti_fname))
    scan.open_label("semantic_kitti/dataset/sequences/{:02d}/labels/{}.label".format(seq_no, kitti_fname))
    scan.colorize()
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(scan.points)
    #pcd.paint_uniform_color([0.6,0.6,0.6])
    pcd.colors = o3d.utility.Vector3dVector(scan.sem_label_color)
    
    road_indices = np.where((scan.sem_label == 40) )#| (scan.sem_label == 44)) #road, parking
    road = o3d.geometry.PointCloud()
    road.points = o3d.utility.Vector3dVector(scan.points[road_indices])
    road.colors = o3d.utility.Vector3dVector(scan.sem_label_color[road_indices])

## Waymo load

In [72]:
frame_name = "39847154216997509_6440_000_6460_000-1568954807924818"
#frame_name = "6503078254504013503_3440_000_3460_000-1557855930472609"
#frame_name = "7240042450405902042_580_000_600_000-1559312894537967"

In [73]:
if not load_kitti:
    #Waymo loading
    with open("waymo/final_road_mask/{}.pkl".format(frame_name), "rb") as f:
        road_points = pickle.load(f)
    road = o3d.geometry.PointCloud()
    road.points = o3d.utility.Vector3dVector(road_points)

    #Waymo loading
    with open("waymo/lidar_data/{}.pkl".format(frame_name), "rb") as f:
        pcd_dict = pickle.load(f)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcd_dict["lidars"]["points_xyz"])

## Point cloud visualization

In [74]:
o3d.visualization.draw_geometries([road], window_name="Road prediction")

In [75]:
o3d.visualization.draw_geometries([pcd], window_name="LiDAR point cloud")

In [76]:
pcd.paint_uniform_color([.6,.6,.6])
road.paint_uniform_color([1,0,0])
o3d.visualization.draw_geometries([pcd, road], window_name="Predicted road in LiDAR PC")

## Transformation to front view

Info: X-axis is in driving direction (depth), Y-Axis orthogonal to the drviing direction (width) and z-axis is height

In [77]:
pcd.get_axis_aligned_bounding_box()

AxisAlignedBoundingBox: min: (-71.9336, -74.4766, -1.44198), max: (76.2153, 74.6406, 5.34238)

In [78]:
max_bounds = pcd.get_max_bound()

In [79]:
min_bounds = pcd.get_min_bound()

In [80]:
front_bb = o3d.geometry.AxisAlignedBoundingBox(np.array([0, min_bounds[1], min_bounds[2]]), max_bounds)

In [81]:
cropped_road = road.crop(front_bb)
cropped_pcd = pcd.crop(front_bb)

In [82]:
o3d.visualization.draw_geometries([cropped_pcd, cropped_road], window_name="Front view cropped PC with pred. road")

## Road prediction ground estimation

In [83]:
ground_model, ground_mask = cropped_road.segment_plane(distance_threshold=0.5,
                                         ransac_n=10,
                                         num_iterations=500)
[a, b, c, d] = ground_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

Plane equation: -0.00x + 0.02y + 1.00z + 0.08 = 0


In [84]:
display_inlier_outlier(cropped_road, ground_mask, window_name="Ground estimation")

Showing outliers (red) and inliers (gray): 


In [85]:
cropped_road = cropped_road.select_by_index(ground_mask)
cropped_road.paint_uniform_color([0.6,0.6,.6])
o3d.visualization.draw_geometries([cropped_road], window_name="Road after ground estimation")

## Remove statistical outliers

In [86]:
cl, ind = cropped_road.remove_statistical_outlier(25,8)

Remove statistical outliers: [========================================] 100%


In [87]:
display_inlier_outlier(cropped_road, ind)

Showing outliers (red) and inliers (gray): 


In [88]:
cropped_road = cropped_road.select_by_index(ind)

In [89]:
o3d.visualization.draw_geometries([cropped_road], window_name="Road after ground estimation")

## Calculate convex hull as approximation for road

In [90]:
hull, _ = cropped_road.compute_convex_hull()
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color((1, 0, 0))
hull.orient_triangles()

True

In [91]:
len(_)

72

In [92]:
hull_points = cropped_road.select_by_index(_).points
hull_points = np.asarray(hull_points)

In [93]:
hull_points[:,-1] = 0
#hull_point

In [94]:
o3d.visualization.draw_geometries([cropped_road, hull], window_name="Front view cropped road and it's convex hull")

In [95]:
hull_min_bound = hull.get_min_bound()
hull_max_bound = hull.get_max_bound()

In [96]:
hull_min_bound[2] = min_bounds[2] #reset z-axis to include objects on road
hull_max_bound[2] = max_bounds[2]

In [97]:
road_bb = o3d.geometry.AxisAlignedBoundingBox(hull_min_bound, hull_max_bound)

In [98]:
bb_cropped_pcd = cropped_pcd.crop(road_bb)

In [99]:
o3d.visualization.draw_geometries([bb_cropped_pcd, hull], window_name="LiDAR PC after bb convex hull (road) cropping")

In [100]:
# from scipy.spatial import ConvexHull, convex_hull_plot_2d
# hull = ConvexHull(np.asarray(flatten_pcd(cropped_road).points)[:,:-1])

In [101]:
# hull_points = np.c_[hull.points, np.zeros(hull.points.shape[0])]
# hull_points

In [102]:
convex_poly = o3d.visualization.SelectionPolygonVolume()
convex_poly.orthogonal_axis = "Z"
convex_poly.axis_max = max_bounds[2]
convex_poly.axis_min = min_bounds[2]
convex_poly.bounding_polygon = o3d.utility.Vector3dVector(hull_points)

In [103]:
convex_poly.axis_max, convex_poly.axis_min

(5.342376708984375, -1.4419784545898438)

In [104]:
cropped_pcd.get_min_bound(), cropped_pcd.get_max_bound()

(array([ 2.44140625e-04, -7.44765625e+01, -1.44197845e+00]),
 array([76.21533203, 74.640625  ,  5.10532379]))

In [105]:
np.asarray(convex_poly.bounding_polygon)

array([[ 68.78027344, -23.44726562,   0.        ],
       [ 68.66992188, -23.58691406,   0.        ],
       [ 68.66943359, -23.765625  ,   0.        ],
       [ 70.17675781, -18.40625   ,   0.        ],
       [ 62.76733398, -13.35009766,   0.        ],
       [ 54.49609375, -19.74121094,   0.        ],
       [ 68.18261719,   5.78613281,   0.        ],
       [ 68.50585938,   5.65380859,   0.        ],
       [ 68.72363281,   5.51074219,   0.        ],
       [ 54.36474609, -24.54248047,   0.        ],
       [ 73.68725586,   2.47851562,   0.        ],
       [ 68.53540039, -12.95751953,   0.        ],
       [ 46.61083984,  -1.34130859,   0.        ],
       [ 66.76098633, -10.53759766,   0.        ],
       [ 54.38623047, -23.79736328,   0.        ],
       [ 54.22875977, -23.87646484,   0.        ],
       [ 53.15283203, -23.98046875,   0.        ],
       [ 66.26489258,  24.70068359,   0.        ],
       [ 70.51025391,  24.45556641,   0.        ],
       [ 72.5090332 ,  23.65478

In [106]:
def flatten_pcd(pcd):
    flattened_pcd = copy.deepcopy(pcd)
    points = np.asarray(flattened_pcd.points)
    points[:,2] = 0
    flattened_pcd.points = o3d.utility.Vector3dVector(points)
    return flattened_pcd

In [107]:
convex_cropped_pcd = convex_poly.crop_point_cloud(flatten_pcd(cropped_pcd))


In [108]:
convex_cropped_pcd.paint_uniform_color([0.5, 0, 0])
o3d.visualization.draw_geometries([pcd, convex_cropped_pcd, hull_ls], window_name="LiDAR PC after bb convex hull (road) cropping")

## Point Cloud distance road objects and ground removal

In [109]:
o3d.visualization.draw_geometries([flatten_pcd(convex_cropped_pcd)], window_name="Flattened convex cropped PC")

In [110]:
distances = np.asarray(flatten_pcd(convex_cropped_pcd).compute_point_cloud_distance(flatten_pcd(cropped_road)))

In [111]:
DISTANCE_OFFSET = 0.2

In [112]:
pc_on_road = convex_cropped_pcd.select_by_index(np.argwhere((distances < DISTANCE_OFFSET) & (distances > 0)))

In [113]:
o3d.visualization.draw_geometries([pc_on_road], window_name="Convex cropped PC on road based on distance to front view cropped road")

In [114]:
def distance_filter(pcd, filter_pcd, offset=0.2, invert=False):
    distances = np.asarray(flatten_pcd(pcd).compute_point_cloud_distance(flatten_pcd(filter_pcd)))
    return pcd.select_by_index(np.argwhere(distances <= offset), invert=invert)

## Ground removal

In [115]:
ground_model, ground_mask = convex_cropped_pcd.segment_plane(distance_threshold=0.5,
                                         ransac_n=20,
                                         num_iterations=500)
[a, b, c, d] = ground_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

Plane equation: -0.00x + -0.00y + 1.00z + -0.00 = 0


In [116]:
display_inlier_outlier(convex_cropped_pcd, ground_mask, window_name="Ground of convex cropped pcd")

Showing outliers (red) and inliers (gray): 


In [117]:
pc_on_road = convex_cropped_pcd.select_by_index(ground_mask, invert=True)
ground = convex_cropped_pcd.select_by_index(ground_mask)

In [118]:
o3d.visualization.draw_geometries([pc_on_road], window_name="Convex cropped PC without estimated ground")

## Clustering

In [119]:
if len(pc_on_road.points) > 0:
    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(
            pc_on_road.cluster_dbscan(eps=1, min_points=50, print_progress=True))

    max_label = labels.max()
    print(f"point cloud has {max_label + 1} clusters")
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pc_on_road.colors = o3d.utility.Vector3dVector(colors[:, :3])
else:
    labels = np.array([])

In [120]:
clustered_obj = pc_on_road.select_by_index(np.argwhere(labels >= 0)) #remove outliers

In [121]:
cropped_road.paint_uniform_color([.6,.6,.6])

PointCloud with 12585 points.

In [122]:
o3d.visualization.draw_geometries([clustered_obj, cropped_road], window_name="Clustered objects on road without ground and within convex road") 

## Mesh calculation

In [123]:
# cropped_road.estimate_normals()

# # estimate radius for rolling ball
# distances = cropped_road.compute_nearest_neighbor_distance()
# avg_dist = np.mean(distances)
# radius = 1.5 * avg_dist   

# mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
#            cropped_road,
#            o3d.utility.DoubleVector([radius, radius * 2]))

# # create the triangular mesh with the vertices and faces from open3d
# tri_mesh = trimesh.Trimesh(np.asarray(mesh.vertices), np.asarray(mesh.triangles),
#                           vertex_normals=np.asarray(mesh.vertex_normals))

# trimesh.convex.is_convex(tri_mesh)

## Voxel downsampling

In [124]:
downsampled_cropped_road = cropped_road.voxel_down_sample(voxel_size=0.5)

In [125]:
#downsampled_cropped_road = cropped_road.random_down_sample(0.1)
downsampled_cropped_road

PointCloud with 1667 points.

In [126]:
o3d.visualization.draw_geometries([downsampled_cropped_road], window_name="Voxel downsampled and front view cropped road")

## PolyLiDAR

## Concave hull 

import alphashape
from descartes import PolygonPatch

points = np.delete(downsampled_cropped_road.points, -1, axis=1)

alpha = 0.95 * alphashape.optimizealpha(points)
hull = alphashape.alphashape(points, alpha)
hull_pts = hull.exterior.coords.xy

fig, ax = plt.subplots()
ax.scatter(hull_pts[0], hull_pts[1], color='red')
ax.add_patch(PolygonPatch(hull, fill=True, color='green'))

from shapely.geometry import Point

alpha_shape = alphashape.alphashape(np.column_stack(hull_pts), 0)

in_hull_mask = np.array([alpha_shape.contains(Point(point[0],point[1])) for point in cropped_pcd.points])

cropped_road.paint_uniform_color([0.6,0.6,0.6])

cropped_pcd.select_by_index(np.argwhere(in_hull_mask))

o3d.visualization.draw_geometries([cropped_pcd.select_by_index(np.argwhere(in_hull_mask)), cropped_road])

## Alphashape surface reconstruction

In [127]:
alpha = 10
print(f"alpha={alpha:.3f}")
downsampled_cropped_road.paint_uniform_color([0.6,0.6,0.6])
road_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(downsampled_cropped_road, alpha)

alpha=10.000


In [128]:
road_mesh.orient_triangles()
o3d.visualization.draw_geometries([road_mesh], mesh_show_back_face=True, window_name="Alphashape of downsampled, front view cropped road")

In [129]:
cropped_road.paint_uniform_color([1,0,0])

PointCloud with 12585 points.

In [130]:
road_mesh.is_watertight()

False

In [131]:
vertices = np.asarray(copy.deepcopy(road_mesh.vertices)) #flatten road mesh
vertices[:,2] = 0
flattend_road_mesh = copy.deepcopy(road_mesh)
flattend_road_mesh.vertices = o3d.utility.Vector3dVector(vertices)

In [132]:
o3d.visualization.draw_geometries([road_mesh, flatten_pcd(cropped_road)], mesh_show_back_face=True, window_name="Comparison of flattened road points and alphashape")

In [133]:
# Create a scene and add the triangle mesh
scene = o3d.t.geometry.RaycastingScene()
flattend_road_mesh = o3d.t.geometry.TriangleMesh.from_legacy(flattend_road_mesh)
_ = scene.add_triangles(flattend_road_mesh)  # we do not need the geometry ID for mesh

In [134]:
def compute_signed_distance_and_closest_goemetry(query_points: np.ndarray):
    closest_points = scene.compute_closest_points(query_points)
    distance = np.linalg.norm(query_points - closest_points['points'].numpy(),
                              axis=-1)
    rays = np.concatenate([query_points, np.ones_like(query_points)], axis=-1)
    intersection_counts = scene.count_intersections(rays).numpy()
    is_inside = intersection_counts % 2 == 1
    distance[is_inside] *= -1
    return distance, closest_points['geometry_ids'].numpy()

In [135]:
distance, closest_points = compute_signed_distance_and_closest_goemetry(np.asarray(flatten_pcd(cropped_pcd).points, dtype=np.float32))

In [136]:
len(cropped_pcd.points), distance.shape, closest_points.shape

(80697, (80697,), (80697,))

In [137]:
on_road_indices =  np.argwhere(distance < 0.0000001)
pc_on_road = cropped_pcd.select_by_index(on_road_indices)

In [138]:
o3d.visualization.draw_geometries([pc_on_road], window_name= "PC on alphashape estimated road")

## Ground removal

In [139]:
ground_model, ground_mask = pc_on_road.segment_plane(distance_threshold=0.2,
                                         ransac_n=10,
                                         num_iterations=300)
[a, b, c, d] = ground_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

Plane equation: -0.00x + 0.02y + 1.00z + 0.06 = 0


In [140]:
display_inlier_outlier(pc_on_road, ground_mask, window_name="Ground removal of PC within road boundaries")

Showing outliers (red) and inliers (gray): 


In [141]:
pc_without_ground = pc_on_road.select_by_index(ground_mask, invert=True)
ground = pc_on_road.select_by_index(ground_mask)

In [142]:
distances = np.asarray(flatten_pcd(pc_without_ground).compute_point_cloud_distance(flatten_pcd(cropped_road)))

In [143]:
DISTANCE_OFFSET = 0
pc_without_road = pc_without_ground.select_by_index(np.argwhere((distances <= DISTANCE_OFFSET)), invert=True)

In [144]:
pc_without_road.paint_uniform_color([1,0,0])
pc_without_ground.paint_uniform_color([0.5,.5,.5])
o3d.visualization.draw_geometries([pc_without_ground, pc_without_road], window_name="Comparison without ground and without road")

In [145]:
cropped_road.paint_uniform_color([0.3,0.3,0.3])
o3d.visualization.draw_geometries([cropped_road, pc_without_road], window_name="Cropped road and PC without road & ground")

In [146]:
len(pc_without_road.points)

3103

## Clustering

In [147]:
if len(pc_without_road.points) > 0:
    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(
            pc_without_road.cluster_dbscan(eps=1, min_points=30, print_progress=True))

    max_label = labels.max()
    print(f"point cloud has {max_label + 1} clusters")
    colors = plt.get_cmap("tab20b")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pc_without_road.colors = o3d.utility.Vector3dVector(colors[:, :3])
else: 
    labels = np.array([])

[Open3D DEBUG] Precompute neighbors.
Precompute neighbors.[================>                       ] 37%                                    ] 5%
[Open3D DEBUG] Compute Clusters
Precompute neighbors.[========================================] 100%
[Open3D DEBUG] Done Compute Clusters: 8            ] 2%
point cloud has 8 clusters====================>    ] 87%


In [148]:
cluster_labels = labels[np.argwhere(labels >= 0)] 
clustered_obj = pc_without_road.select_by_index(np.argwhere(labels >= 0))

In [149]:
o3d.visualization.draw_geometries([clustered_obj], window_name="Clustered object on alhphashape road without ground & ground prediction")

In [150]:
vertices = np.asarray(copy.deepcopy(road_mesh.vertices)) #flatten road mesh
vertices[:,2] -= 1
shifted_road_mesh = copy.deepcopy(road_mesh)
shifted_road_mesh.vertices = o3d.utility.Vector3dVector(vertices)
shifted_road_mesh.paint_uniform_color([0.9, 0.9,0.9])

TriangleMesh with 1420 points and 3224 triangles.

In [151]:
pcd.paint_uniform_color([0.8,0.8,0.8])
o3d.visualization.draw_geometries([ pcd, cropped_road, clustered_obj], width=3072, height=1920, window_name="Result: Clusters on road ahead")

In [152]:
clusters = np.hstack((np.asarray(clustered_obj.points), cluster_labels))
clusters.shape

(2731, 4)

In [153]:
labeled_clusters= np.c_[clusters, -np.ones(clusters.shape[0])]
labeled_clusters

array([[ 55.21484375, -13.83984375,   4.10257721,   0.        ,
         -1.        ],
       [ 55.22167969, -13.9765625 ,   4.10461426,   0.        ,
         -1.        ],
       [ 55.02026367, -14.33105469,   4.10283661,   0.        ,
         -1.        ],
       ...,
       [ 13.07663059,  -3.81503773,   0.44700998,   7.        ,
         -1.        ],
       [ 13.0859642 ,  -2.94334888,   0.36902636,   7.        ,
         -1.        ],
       [ 13.05995464,  -3.80804634,   0.34077984,   7.        ,
         -1.        ]])

## Create AABBs

In [154]:
sorted_clusters = clusters[clusters[:,-1].argsort()]

In [155]:
cluster_list = np.split(sorted_clusters[:,:-1], np.unique(sorted_clusters[:,-1], return_index=True)[1][1:])

In [156]:
cluster_list[3]

array([[47.65600586, -3.45703125,  0.92795563],
       [47.67553711, -3.67822266,  0.92784119],
       [47.6784668 , -3.12841797,  0.92665863],
       [47.92553711, -2.92382812,  0.91945648],
       [48.70776367, -2.97558594,  0.30427551],
       [47.67895508, -3.78857422,  0.92795563],
       [47.6875    , -3.89990234,  0.92790985],
       [47.67871094, -4.00878906,  0.92832947],
       [47.67211914, -3.56835938,  0.92772675],
       [47.65991211, -3.23681641,  0.92740631],
       [47.68164062, -4.01123047,  0.77368927],
       [48.19873047, -3.83349609,  0.47493744],
       [47.69726562, -4.78271484,  1.33036804],
       [47.70214844, -4.67285156,  1.33003235],
       [47.56835938, -3.77978516,  0.65014648],
       [47.5300293 , -3.66796875,  0.65128326],
       [47.57324219, -3.56152344,  0.64969635],
       [47.52270508, -3.44775391,  0.65120697],
       [47.61206055, -3.34472656,  0.64806366],
       [47.60766602, -3.234375  ,  0.64801788],
       [47.6262207 , -3.12548828,  0.647

In [157]:
cluster_bboxes = [o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster)) for cluster in cluster_list]

In [158]:
cluster_bboxes

[AxisAlignedBoundingBox: min: (53.6423, -24.6396, 0.533463), max: (56.562, -12.9565, 4.11474),
 AxisAlignedBoundingBox: min: (57.2004, -14.0854, 0.703117), max: (64.1772, -11.9731, 3.78047),
 AxisAlignedBoundingBox: min: (46.3843, -1.34814, 0.345856), max: (47.6956, 0.482422, 1.86856),
 AxisAlignedBoundingBox: min: (46.8838, -4.78271, 0.304276), max: (49.176, -2.92383, 1.46764),
 AxisAlignedBoundingBox: min: (62.187, 8.20117, 0.0645981), max: (63.6711, 10.1797, 1.33728),
 AxisAlignedBoundingBox: min: (31.7302, -4.49658, 0.284782), max: (34.8674, -2.6543, 1.59477),
 AxisAlignedBoundingBox: min: (29.5779, -7.7085, 1.00103), max: (31.9309, -6.23584, 1.57664),
 AxisAlignedBoundingBox: min: (12.6792, -4.41846, 0.262344), max: (16.5796, -2.65479, 1.75541)]

## Sanity check

In [159]:
ON_ROAD_POINTS = 5

In [160]:
road_z_mean = cropped_road.compute_mean_and_covariance()[0][2]
road_z_mean 

-0.0009998581406612647

In [161]:
assert len(cluster_bboxes) == len(cluster_list)
checked_cluster_bboxes = []
checked_cluster_list = []
for i in range(len(cluster_bboxes)):
    expanded_cluster_bbox = copy.deepcopy(cluster_bboxes[i])
    bbox_z_center = expanded_cluster_bbox.get_center()[2]
    expanded_cluster_bbox.translate(np.array([0, 0 , (road_z_mean - bbox_z_center)/2]))
    if(len(expanded_cluster_bbox.get_point_indices_within_bounding_box(o3d.utility.Vector3dVector(cropped_road.points))) > ON_ROAD_POINTS):
        print(len(expanded_cluster_bbox.get_point_indices_within_bounding_box(o3d.utility.Vector3dVector(cropped_road.points))))
        checked_cluster_bboxes.append(cluster_bboxes[i])
        checked_cluster_list.append(cluster_list[i])

373
15
6
7
18
153


In [163]:
checked_cluster_bboxes

[AxisAlignedBoundingBox: min: (53.6423, -24.6396, 0.533463), max: (56.562, -12.9565, 4.11474),
 AxisAlignedBoundingBox: min: (57.2004, -14.0854, 0.703117), max: (64.1772, -11.9731, 3.78047),
 AxisAlignedBoundingBox: min: (46.3843, -1.34814, 0.345856), max: (47.6956, 0.482422, 1.86856),
 AxisAlignedBoundingBox: min: (46.8838, -4.78271, 0.304276), max: (49.176, -2.92383, 1.46764),
 AxisAlignedBoundingBox: min: (31.7302, -4.49658, 0.284782), max: (34.8674, -2.6543, 1.59477),
 AxisAlignedBoundingBox: min: (12.6792, -4.41846, 0.262344), max: (16.5796, -2.65479, 1.75541)]

In [164]:
cluster_bboxs_dims =[np.concatenate([cluster_bbox.get_center(), cluster_bbox.get_extent()]) for cluster_bbox in checked_cluster_bboxes]
if len(cluster_bboxs_dims) > 0:
    cluster_bboxs_dims = np.vstack(cluster_bboxs_dims)
else: 
    cluster_bboxs_dims = np.array(cluster_bboxs_dims)
cluster_bboxs_dims

array([[ 55.10217285, -18.7980957 ,   2.32410049,   2.91967773,
         11.68310547,   3.58127594],
       [ 60.68884277, -13.02929688,   2.24179459,   6.97680664,
          2.11230469,   3.07735443],
       [ 47.03991699,  -0.43286133,   1.10720825,   1.3112793 ,
          1.83056641,   1.52270508],
       [ 48.02990723,  -3.85327148,   0.88595581,   2.29223633,
          1.85888672,   1.1633606 ],
       [ 33.29882812,  -3.57543945,   0.93977737,   3.13720703,
          1.84228516,   1.30998993],
       [ 14.62939453,  -3.53662109,   1.0088768 ,   3.90039062,
          1.76367188,   1.49306488]])

In [165]:
labeled_cluster_bboxs_dims = np.c_[cluster_bboxs_dims, -np.ones(cluster_bboxs_dims.shape[0])]
labeled_cluster_bboxs_dims

array([[ 55.10217285, -18.7980957 ,   2.32410049,   2.91967773,
         11.68310547,   3.58127594,  -1.        ],
       [ 60.68884277, -13.02929688,   2.24179459,   6.97680664,
          2.11230469,   3.07735443,  -1.        ],
       [ 47.03991699,  -0.43286133,   1.10720825,   1.3112793 ,
          1.83056641,   1.52270508,  -1.        ],
       [ 48.02990723,  -3.85327148,   0.88595581,   2.29223633,
          1.85888672,   1.1633606 ,  -1.        ],
       [ 33.29882812,  -3.57543945,   0.93977737,   3.13720703,
          1.84228516,   1.30998993,  -1.        ],
       [ 14.62939453,  -3.53662109,   1.0088768 ,   3.90039062,
          1.76367188,   1.49306488,  -1.        ]])

In [166]:
if not load_kitti:
    cluster_dir = "waymo/clusters"
    if not os.path.exists(cluster_dir):
        os.makedirs(cluster_dir)
    with open('waymo/clusters/{}.pkl'.format(frame_name), 'wb') as f:
        pickle.dump({"cluster_pcs": labeled_clusters, "cluster_bboxes": labeled_cluster_bboxs_dims},f)

In [167]:
if load_kitti:
    cluster_dir = "semantic_kitti/dataset/sequences/{:02d}/clusters".format(seq_no)
    if not os.path.exists(cluster_dir):
        os.makedirs(cluster_dir)
    with open(os.path.join(cluster_dir, '{}.pkl'.format(kitti_fname)), 'wb') as f:
        pickle.dump({"cluster_pcs": labeled_clusters, "cluster_bboxes": labeled_cluster_bboxs_dims},f)

## Visualize clustering

In [168]:
sorted_clusters = clusters[clusters[:,-1].argsort()]
np.unique(sorted_clusters[:,-1])
cluster_list = np.split(sorted_clusters[:,:-1], np.unique(sorted_clusters[:,-1], return_index=True)[1][1:])

In [169]:
bboxes = [o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster)) for cluster in cluster_list]

In [170]:
#Visualize sanity check
road_z_mean = cropped_road.compute_mean_and_covariance()[0][2]
translated_cluster_line_sets = []
for box in bboxes:
    bbox_z_center = box.get_center()[2]
    translated_box = copy.deepcopy(box)
    translated_box.translate(np.array([0, 0 , (road_z_mean - bbox_z_center)]))
    translated_cluster_line_sets.append(o3d.geometry.LineSet().create_from_axis_aligned_bounding_box(translated_box))
for lineset in translated_cluster_line_sets:
    lineset.colors = o3d.utility.Vector3dVector([[0.6, 0.6,0.6] for i in range(12)])

In [171]:
cluster_line_sets = [o3d.geometry.LineSet().create_from_axis_aligned_bounding_box(box) for box in bboxes]
for lineset in cluster_line_sets:
    lineset.colors = o3d.utility.Vector3dVector([[255, 0,0] for i in range(12)])

In [172]:
pcd.paint_uniform_color([0.8,0.8,0.8])
visuals = [pcd, cropped_road, clustered_obj]
visuals += cluster_line_sets
visuals += translated_cluster_line_sets
o3d.visualization.draw_geometries(visuals, width=3072, height=1920, window_name="Result: Clusters on road ahead",  
			 front= [ -0.84376060007152665, -0.075611776213210732, 0.53136701917302009  ],
			 lookat =  [ 4.0740218605326444, 1.3355749707693445, 3.0878925453263633 ],
			 up =  [0.5306174540674945, 0.03135054190208468, 0.84703144036179656  ],
			 zoom = 0.042000000000000016)